# EMNIST-ByClass Handwritten Character Recognizer

## Setup

In [ ]:
!git clone https://github.com/XavierSpycy/EMNIST-Classifier.git
%cd EMNIST-Classifier

In [ ]:
import torch
from torch.utils.data import DataLoader
from charclf.dataset import load_dataset
from charclf.models import VGGNet, AlexNet, SpinalNet, ResNet
from charclf.tools.eval import multi_evaluate, evaluate, confusion_matrix_viz
from charclf.tools.viz import predict
from charclf.tools.train import train_

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Performance on Our Datasets

In [ ]:
emnist_train, emnist_test = load_dataset('eval')
train_loader = DataLoader(emnist_train, batch_size=100, shuffle=False, num_workers=2)
test_loader = DataLoader(emnist_test, batch_size=100, shuffle=False, num_workers=2)

In [ ]:
alexnet = AlexNet()
alexnet.to(device).load_state_dict(torch.load("model_hub/fine_tuned//alexnet_tuned.pth", map_location=device))
vggnet = VGGNet()
vggnet.to(device).load_state_dict(torch.load("model_hub/fine_tuned/vggnet_tuned.pth", map_location=device))
spinalnet = SpinalNet()
spinalnet.to(device).load_state_dict(torch.load("model_hub/fine_tuned/spinalnet_tuned.pth", map_location=device))
resnet = ResNet()
resnet.to(device).load_state_dict(torch.load("model_hub/fine_tuned/resnet_tuned.pth", map_location=device))

In [ ]:
models = [alexnet, vggnet, spinalnet, resnet]
multi_evaluate(models, train_loader, device)

In [ ]:
multi_evaluate(models, test_loader, device)

In [ ]:
predict(vggnet, test_loader, device)

## Performance on the Entire Datasets

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets

transform = transforms.Compose([transforms.Resize(28),
                                transforms.ToTensor(),
                                transforms.Normalize((0.1736,), (0.3248,)),
                                ])

emnist_train_complete = datasets.EMNIST(root='./EMNIST', split='byclass', train=True, download=True, transform=transform)
emnist_test_complete = datasets.EMNIST(root='./EMNIST', split='byclass', train=False, download=True, transform=transform)

In [ ]:
complete_train = DataLoader(emnist_train_complete, batch_size=100, shuffle=False, num_workers=2)
complete_test = DataLoader(emnist_test_complete, batch_size=100, shuffle=False, num_workers=2)
acc_train, precision_train, recall_train, f1_train, cm_train = evaluate(vggnet, complete_train)
acc_test, precision_test, recall_test, f1_test, cm_test = evaluate(vggnet, complete_test)
print(f"Accuracy on the training set: {acc_train*100:.2f}%.")
print(f"Precision on the training set: {precision_train*100:.2f}%.")
print(f"Recall on the training set: {recall_train*100:.2f}%.")
print(f"F1 Score on the training set: {f1_train*100:.2f}%.")
print(f"Accuracy on the test set: {acc_test*100:.2f}%.")
print(f"Precision on the test set: {precision_test*100:.2f}%.")
print(f"Recall on the test set: {recall_test*100:.2f}%.")
print(f"F1 Score on the test set: {f1_test*100:.2f}%.")
confusion_matrix_viz(cm_train)
confusion_matrix_viz(cm_test)

## Performance on the Downstream Task

In [ ]:
import torch.nn as nn
import torch.optim as optim

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1325, ), (0.3039, ))])
mnist_train = datasets.MNIST(root='./MMNIST', train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(root='./MNIST', train=False, download=True, transform=transform)

vggnet = VGGNet().to(device)
vggnet.load_state_dict(torch.load("model_hub/fine_tuned/vggnet_tuned.pth", map_location=device))
vggnet.classifier[8] = torch.nn.Linear(512, 10)
vggnet.to(device)
for name, param in vggnet.named_parameters():
    if 'classifier' not in name:
        param.requires_grad = False
train_loader = DataLoader(mnist_train, batch_size=32, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=100, shuffle=False)
optimizer = optim.SGD(vggnet.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()
epochs = 5
train_(vggnet, train_loader, optimizer, criterion, epochs)

In [ ]:
acc_train, precision_train, recall_train, f1_train, cm_train = evaluate(vggnet, train_loader)
acc_test, precision_test, recall_test, f1_test, cm_test = evaluate(vggnet, test_loader)
print(f"Accuracy on the training set: {acc_train*100:.2f}%.")
print(f"Precision on the training set: {precision_train*100:.2f}%.")
print(f"Recall on the training set: {recall_train*100:.2f}%.")
print(f"F1 Score on the training set: {f1_train*100:.2f}%.")
print(f"Accuracy on the test set: {acc_test*100:.2f}%.")
print(f"Precision on the test set: {precision_test*100:.2f}%.")
print(f"Recall on the test set: {recall_test*100:.2f}%.")
print(f"F1 Score on the test set: {f1_test*100:.2f}%.")